<a href="https://colab.research.google.com/github/tomywe/Facial-Keypoints/blob/master/RADLogics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import sklearn
import zipfile

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torchvision.transforms as T

import cv2

from google.colab import files
import io

import os

In [129]:
os.listdir()

['.config',
 'tiny-imagenet-200.zip',
 'tiny-imagenet-200',
 'alldata',
 'sample_data']

In [56]:
# upload tiny-imagenet-200 and lision data

uploaded = files.upload()

Saving tiny-imagenet-200.zip to tiny-imagenet-200.zip


In [0]:
# extract images from zip files to directoris 

with zipfile.ZipFile('alldata.zip', 'r') as zipobj:
  zipobj.extractall(path='alldata')
with zipfile.ZipFile('tiny-imagenet-200.zip', 'r') as zipobj:
  zipobj.extractall()

In [0]:
device = torch.device('cpu')
if torch.cuda.is_available():
  device = torch.device('cuda')

In [0]:
class LisionData(Dataset):
  
  def __init__(self, transform=None):
    im_paths = os.listdir('alldata')
    self.paths = im_paths
    self.labels = np.array([self.get_label(path) for path in im_paths])
    self.transform = transform
    
  def __len__(self):
    return len(self.paths)
    
  def get_label(self, path):
    path_split = path.split('_')
    c = path_split[3].split('.')[0]
    c = int(c)
    return c
  
  def __getitem__(self, index):
    im = cv2.imread('alldata' + self.paths[index])
    im = cv2.resize(im, (64,64), interpolation=cv2.INTER_CUBIC)
    label = self.labels[index]
    if self.transform is not None:
      im = self.transform(im)
      im = im.to(device)
    return im, torch.tensor(label)

In [0]:
def create_k_fold_indexes(r, k=5):
  indexes_lists = []
  for _ in range(k):
    indexes_lists.append([])
  indexes = list(range(r))
  i = 0
  while len(indexes) > 0:
    random_index = np.random.randint(len(indexes))
    indexes_lists[i%k].append(indexes.pop(random_index))
    i += 1
  return indexes_lists

lists = create_k_fold_indexes(182)
test_indexes = lists.pop()
val_indexes = lists.pop()
train_indexes = []
while len(lists) > 0:
  train_indexes += lists.pop()

In [0]:
class TinyImageData(Dataset):
  
  def __init(self, transform=None, val=False):
    image_names = []
    labels = []
    self.root_dir = 'tiny-imagenet-200/'
    for d in os.listdir(self.root_dir):
      train_names += os.listdir(self.root_dir 'train/' + d + '/images')
      labels.append(dir)
    data_table = pd.read_csv(self.root_dir + 'val/' + 'val_annotations.txt',
                             sep='\t', header=None)
    val_names = pd[0]
    val_labels = pd[1]
    self.train_names = train_names
    self.val_names = val_names
    self.labels = labels
    self.val_labels = val_labels
    self.transform = transform
    self.len = len(train_names)
    if val:
      self.len = len(val_names)
    self.val = val
    
  def __len__(self):
    return self.len
  
  def __getitem__(self.index):
    if not self.val:
      name = self.train_names[index]
      name_split = name.split('_')
      label = self.labels.index(name_split[0])
      path = self.root_dir + 'train/' + self.labels[label] + '/images' + name
    else:
      name = self.val_names[index]
      label = self.labels.index(self.val_labels[index])
      path = self.root_dir + 'val/' + '/images' + name
    im = cv2.imread(path)
    if self.transform:
      im = transform(im)
    return im, torch.tensor(label)

In [0]:
norm_set = LisionData(transform=T.ToTensor())
norm_loader = DataLoader(norm_set, batch_size=182)

def compute_mean_std(loader):
    mean = 0.0
    var = 0.0
    for images, _ in loader:
        N, C, H, W = images.size()
        images = images.view(N, C, -1)
        mean += images.mean(2).sum(0)
        var += ((images - mean.unsqueeze(1))**2).sum([0,2])
    mean = mean / len(loader.dataset)
    std = torch.sqrt(var / (len(loader.dataset)*H*W))
    return mean, std

mean, std = compute_mean_std(norm_loader)

print(mean, std)

tensor([0.6058, 0.6058, 0.6058], device='cuda:0') tensor([109.6499, 109.6499, 109.6499], device='cuda:0')


In [0]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize(mean, std)])



In [0]:
def create_kfold_indexes(k, n_samples)
  l = np.arange(n_samples)
  a = np.random.choice(l, size=(,3), replace=False)
  mask = np.ones(l.shape, dtype=bool)
  mask[a.reshape(-1)] = False
  last = l[mask]

In [0]:
class Flatten(mm.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class k_fold_model(nn.Module):
  def __init__():
    super(k_fold_model, self).__init()
    self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
    self.norm1 = nn.BatchNorm2d(32)
    self.conv2_1 = nn.Conv2d(32, 64, 3, padding=1)
    self.norm2_1 = nn.BatchNorm2d(64)
    self.conv2_2 = nn.Conv2d(64, 64, 3, padding=1)
    self.nrom2_2 = nn.BatchNorm2d(64)
    self.conv3_1 = nn.Conv2d(64, 128, 3, padding=1)
    self.nrom3_1 = nn.BatchNorm2d(128)
    self.conv3_2 = nn.Conv2d(128, 128, 3, padding=1)
    self.nrom3_2 = nn.BatchNorm2d(128)
    self.conv4_1 = nn.Conv2d(128, 256, 3, 1)
    self.nrom4_1 = nn.BatchNorm2d(256)
    self.conv4_2 = nn.Conv2d(256, 16, 3, 1)
    self.nrom4_2 = nn.BatchNorm2d(16)
    self.fc1 = nn.Linear(4*4*16, 100)
    self.fc2 = nn.Linear(100, 3)
    self.pool = nn.MaxPool2d(2)
    
  def forward(self, x):
    # 1st conv module, input 64x64x3, output 32x32x64
    x = self.conv1(x)
    x = self.norm1(x)
    x = self.pool(x)
    x = F.relu(x)
    # 2nd conv module, input 32x32x64, output 16x16x128
    x = self.conv2_1(x)
    x = self.norm2_1(x)
    x = F.relu(x)
    x = self.conv2_2(x)
    x = self.norm2_2(x)
    x = self.pool(x)
    x = F.relu(x)
    # 3rd conv module, input 16x16x128, output 4x4x256
    x = self.conv3_1(x)
    x = self.norm3_1(x)
    x = F.relu(x)
    x = self.conv3_2(x)
    x = self.norm3_1(x)
    x = self.pool(x)
    x = F.relu(x)
    # 4th conv module, input 4x4x256, output 4x4x16
    x = self.conv4_1(x)
    x = self.norm4_1(x)
    x = F.relu(x)
    x = self.conv4_2(x)
    x = self.norm4_2(x)
    x = F.relu(x)
    # Fully connected layers to classifier
    # 1st fc layer, input 256 output 100
    x = Flatten(x)
    x = self.fc1(x)
    x = F.dropout2d(x)
    x = self.relu(x)
    # 2nd fc layer, input 100, output 3
    x = self.fc2(x)
    return x

In [0]:
resnet18 = models.resnet18()
resnet18.fc = nn.Linear(512, 200)

In [0]:
def train(model, optimizer, loader, val_loader, epochs=25):
  train_loss = []
  val_loss
  for e in range(epochs):
    for t, (data, labels) in enumerate(loader):
      model.train()
      scores = model(data)
      loss = F.cross_entropy(scores, labels)
      optimizer.zero_grad()
      loss.backward()
      oprimizer.step()
      train_loss.append(loss)
      # Print progress
      if t%10 = 0:
        print('ephoch {0} iteration {1} train loss {2}'.format(e, t, loss.item))
        val_loss.append(accuracy(model, val_loader))
        print()
  return train_loss, val_loss

def accuracy(model, loader):
  loss_list = []
  model.eval()
  num_correct = 0
  num_samples = 0
  with torch.no_grad():
    for data, labels in loader:
      scores = model(data)
      loss_list.append(F.cross_entropy(scores, labels))
      _, preds = scores.argmax(1)
      num_correct += (preds == labels).sum()
      num_samples += len(labels)
    acc = float(num_correct) / num_samples
    print('got {0} / {1} correct at {2}%'.format(num_correct, num_samples, acc)
    return np.mean(loss_list)

In [0]:
def find_lr(model_class, train_loader, val_loader):
  best_model = None
  best_lr = np.ln(3)
  for lr in [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]:
    model = model_class()
    optimizer = optim.Adam(model.parameters, lr=lr)
    train_loss, val_loss = train(model, optimizer, train_loader, val_loader, epochs=5)
    if train_loss[-1] < best_loss:
      best_loss = loss[-1]
      best_lr = lr
  return best_lr

best_lr = find_lr(k_fold_model, train_loader, val_loader)

In [15]:
model_list = []
optimizer_list = []
for _ in range(k):
  model_list.append(k_fold_model())
  optimizer_list.append(optim.Adam(model_list[i].parameters, lr=best_lr))
  train(model_list[-1], optimizer_list[-1], train_loader, val_loader, epochs=50)
  
test_list = []
for model in model_list:
  test_list.append(model(test_data))
test_result = np.mean(test_list, axis=0)

NameError: ignored

In [0]:
lr_resnet = find_lr(resnet18, )
train(resnet18, res_optim, res_train_loader, res_val_loader)

plt.imshow(train_loss)
plt.imshow(val_loss)

In [106]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [0]:
resnet18.fc = nn.Linear(512, 3)